In [ ]:
""" Functions for generating 2D grid maps, for AI Lab 2 - path planning.
"""

import random
import numpy as np
import matplotlib.pyplot as plt

percentOfObstacle = 0.9  # 30% - 60%, random

def generateMap2d(size_):

    '''Generates a random 2d map with obstacles (small blocks) randomly distributed. 
       You can specify any size of this map but your solution has to be independent of map size

    Parameters:
    -----------
    size_ : list
        Width and height of the 2d grid map, e.g. [60, 60]. The height and width of the map shall be greater than 20.

    Returns:
    --------
        map2d : array-like, shape (size_[0], size_[1])
           A 2d grid map, cells with a value of 0: Free cell; 
                                                -1: Obstacle;
                                                -2: Start point;
                                                -3: Goal point;
    '''
    
    size_x, size_y = size_[0], size_[1]

    map2d = np.random.rand(size_y, size_x)
    perObstacles_ = percentOfObstacle
    map2d[map2d <= perObstacles_] = 0
    map2d[map2d > perObstacles_] = -1

    yloc, xloc = [np.random.random_integers(0, size_x-1, 2), np.random.random_integers(0, size_y-1, 2)]
    while (yloc[0] == yloc[1]) and (xloc[0] == xloc[1]):
        yloc, xloc = [np.random.random_integers(0, size_x-1,2), np.random.random_integers(0, size_y-1, 2)]

    map2d[xloc[0]][yloc[0]] = -2
    map2d[xloc[1]][yloc[1]] = -3
    
    print('start',map2d[xloc[0]][yloc[0]], 'Goal', map2d[xloc[1]][yloc[1]]  )
    print('start',xloc[0], yloc[0], 'Goal', xloc[1], yloc[1])
    

    return map2d

In [ ]:
# Generate 2d grid map with rotated-H-shape object
def generateMap2d_obstacle(size_):
    '''Generates a random 2d map with a rotated-H-shape object in the middle and obstacles (small blocks) randomly distributed. 
       You can specify any size of this map but your solution has to be independent of map size

    Parameters:
    -----------
    size_ : list
        Width and height of the 2d grid map, e.g. [60, 60]. The height and width of the map shall be greater than 40.

    Returns:
    --------
        map2d : array-like, shape (size_[0], size_[1])
           A 2d grid map, cells with a value of 0: Free cell; 
                                               -1: Obstacle;
                                               -2: Start point;
                                               -3: Goal point;
                                            
       [ytop, ybot, minx] : list
           information of the rotated-H-shape object
           ytop - y coordinate of the top horizontal wall/part
           ybot - y coordinate of the bottom horizontal wall/part
           minx - X coordinate of the vertical wall 
    '''
    
    size_x, size_y = size_[0], size_[1]
    map2d = generateMap2d(size_)

    map2d[map2d==-2] = 0
    map2d[map2d==-3] = 0

    # add special obstacle
    xtop = [np.random.random_integers(5, 3*size_x//10-2), np.random.random_integers(7*size_x//10+3, size_x-5)]
    ytop = np.random.random_integers(7*size_y//10 + 3, size_y - 5)
    xbot = np.random.random_integers(3, 3*size_x//10-5), np.random.random_integers(7*size_x//10+3, size_x-5)
    ybot = np.random.random_integers(5, size_y//5 - 3)


    map2d[ybot, xbot[0]:xbot[1]+1] = -1
    map2d[ytop, xtop[0]:xtop[1]+1] = -1
    minx = (xbot[0]+xbot[1])//2
    maxx = (xtop[0]+xtop[1])//2
    if minx > maxx:
        tempx = minx
        minx = maxx
        maxx = tempx
    if maxx == minx:
        maxx = maxx+1

    map2d[ybot:ytop, minx:maxx] = -1
    startp = [np.random.random_integers(0, size_x//2 - 4), np.random.random_integers(ybot+1, ytop-1)]

    map2d[startp[1], startp[0]] = -2
    goalp = [np.random.random_integers(size_x//2 + 4, size_x - 3), np.random.random_integers(ybot+1, ytop-1)]

    map2d[goalp[1],goalp[0]] = -3
    #return map2d, [startp[1], startp[0]], [goalp[1], goalp[0]], [ytop, ybot]
    return map2d, [ytop, ybot, minx]

In [ ]:
# helper function for plotting the result
def plotMap(map2d_, path_=None, title_ =''):
    
    '''Plots a map (image) of a 2d matrix with a path from start point to the goal point. 
        cells with a value of 0: Free cell; 
                             -1: Obstacle;
                             -2: Start point;
                             -3: Goal point;
    Parameters:
    -----------
    map2d_ : array-like
        an array with Real Numbers
        
    path_ : array-like
        an array of 2d corrdinates (of the path) in the format of [[x0, y0], [x1, y1], [x2, y2], ..., [x_end, y_end]]
        
    title_ : string
        information/description of the plot

    Returns:
    --------

    '''
    
    import matplotlib.cm as cm
    plt.interactive(False)
    
    colors_nn = int(map2d_.max())
    colors = cm.winter(np.linspace(0, 1, colors_nn))

    colorsMap2d = [[[] for x in range(map2d_.shape[1])] for y in range(map2d_.shape[0])]
    # Assign RGB Val for starting point and ending point
    locStart, locEnd = np.where(map2d_ == -2), np.where(map2d_ == -3)
    
    
    colorsMap2d[locStart[0][0]][locStart[1][0]] = [.0, .0, .0, 1.0]  # black
    colorsMap2d[locEnd[0][0]][locEnd[1][0]] = [0.8, 0.4, 0.1, 0.5]  # brown

    # Assign RGB Val for obstacle
    locObstacle = np.where(map2d_ == -1)
    for iposObstacle in range(len(locObstacle[0])):
        colorsMap2d[locObstacle[0][iposObstacle]][locObstacle[1][iposObstacle]] = [1.0, .0, .0, 1.0]
    # Assign 0
    locZero = np.where(map2d_ == 0)

    for iposZero in range(len(locZero[0])):
        colorsMap2d[locZero[0][iposZero]][locZero[1][iposZero]] = [1.0, 1.0, 1.0, 1.0]

    # Assign Expanded nodes
    locExpand = np.where(map2d_>0)

    for iposExpand in range(len(locExpand[0])):
        _idx_ = int(map2d_[locExpand[0][iposExpand]][locExpand[1][iposExpand]]-1)
        colorsMap2d[locExpand[0][iposExpand]][locExpand[1][iposExpand]] = colors[_idx_]

    for irow in range(len(colorsMap2d)):
        for icol in range(len(colorsMap2d[irow])):
            if colorsMap2d[irow][icol] == []:
                colorsMap2d[irow][icol] = [1.0, 0.0, 0.0, 1.0]
                
    if  path_ is not None: path = path_.T.tolist()
    
    plt.figure()
    plt.title(title_)
    plt.imshow(colorsMap2d, interpolation='nearest')
    plt.colorbar()
    if  path_ is not None:plt.plot(path[:][0],path[:][1], color='magenta',linewidth=2.5)
    plt.show()



In [ ]:
# import numpy as np
import math
import heapq

# Priority Queue based on heapq
class PriorityQueue:
    def __init__(self):
        self.elements = []
    def isEmpty(self):
        return len(self.elements) == 0
    def add(self, priority, item):
        heapq.heappush(self.elements,(priority,item))
    def remove(self):
        return heapq.heappop(self.elements)

In [ ]:
def get_neighbors(current, goal_position,map1, heuristic_type):
    x,y =current
    adjacent_cells=[]
    
    shift = [[-1, 0 ], # go up
            [ 0, -1], # go left
            [ 1, 0 ], # go down
            [ 0, 1 ]] # go right
    
    if heuristic_type=='manhattan':
        
        for i in range(len(shift)):
            x1 = x + shift[i][0]
            y1 = y + shift[i][1]
            distx= abs(goal_position[0] -x1)
            disty= abs(goal_position[1]-y1)
            heuristic= abs(distx) + abs(disty)
        
            #check if outside the boundary
            if x1 >= 0 and x1 < len(map1) and y1 >=0 and y1 <len(map1[0]):
                #move to adjacent cell if only its value is zero
                if map1[x1][y1] == 0 or map1[x1][y1] == -3 :
                    adjacent_cells.append((heuristic, (x1, y1)))
                
    elif heuristic_type=='euclidian':
        for i in range(len(shift)):
            x1 = x + shift[i][0]
            y1 = y + shift[i][1]
            distx= goal_position[0] -x1
            disty= goal_position[1]-y1
            heuristic= math.sqrt(distx**2 + disty**2)
        
            #check if outside the boundary
            if x1 >= 0 and x1 < len(map1) and y1 >=0 and y1 <len(map1[0]):
                #move to adjacent cell if only its value is zero
                if map1[x1][y1] == 0 or map1[x1][y1] == -3 :
                    adjacent_cells.append((heuristic, (x1, y1)))
                
    
    
    
    return adjacent_cells


In [ ]:
# An example of search algorithm, feel free to modify and implement the missing part

def greedy_search(map1, heuristic_type):

    exp_map = np.copy(map1)
    for row in range(len(map1)):
        for column in range(len(map1)):
            if map1[row][column]==-2:
                xs, ys= row,column
                
            if map1[row][column]==-3:
                xg, yg= row,column
                
                
    
    start_position=(xs, ys)
    goal_position=(xg, yg)
    
    if heuristic_type=='manhattan':
        #heuristic = f(n)=h(n)
        distx= abs(goal_position[0]-start_position[0])
        disty= abs(goal_position[1]-start_position[1])
        heuristic= abs(distx) + abs(disty)
        
    elif heuristic_type=='euclidian':
        distx= goal_position[0]-start_position[0]
        disty= goal_position[1]-start_position[1]
        heuristic= math.sqrt(distx**2 + disty**2)
        
    
    # set to store visited cells
    visited=set()
    visited_value=(heuristic,start_position)
    
    visited.add(visited_value)
    
    # Boolean found if goal is reached
    found=False

    # priority queue
    frontier = PriorityQueue()
    
    # path taken
    came_from=dict()

    # add starting cell to priority queue
    frontier.add(heuristic, start_position)
    
    #cost is the total number of nodes expanded
    cost=0

    
    while not frontier.isEmpty():
        h_cost, current = frontier.remove()


        # check if the goal is reached
        if current == goal_position:
            found=True
            break
 
        # for each neighbour of the current cell return expanded adjacent cells
        # (avoid repetitions by using set visited)
        cells=get_neighbors(current,goal_position, map1, heuristic_type)
        for next1 in cells:
            if next1 not in visited:
                visited.add(next1)
                came_from[(xs,ys)]=tuple(current)
                heuristic,(x,y)=next1
                
                #increment cost
                cost+=1
                if exp_map[x][y]!=-3:exp_map[x][y]=cost

                # add next cell to priority list
                frontier.add(heuristic, (x,y)) ###Counter check which is the priority and item.
                
                #add to path
                came_from[(x,y)]=tuple(current)
    
    
    
    if found==False: 
        print("No path to goal")
        return -1
    else:
        a,b=goal_position
        goal_position2=b,a
        reverse_path = [goal_position2]
        while tuple(goal_position) != tuple(start_position):
            goal_position = came_from[tuple(goal_position)]
            x2,y2=goal_position
            reverse_path.append((y2,x2))
            path2=list(reversed(reverse_path))
        
        path1=np.array(path2)
        plotMap(map1)
        plotMap(exp_map, path1,'path')
        return cost, len(path1)

In [ ]:
map1 = generateMap2d([60, 60])
greedy_search(map1, 'manhattan')

In [ ]:
map1 = generateMap2d([60, 60])
greedy_search(map1, 'euclidian')

In [ ]:
map2, info=generateMap2d_obstacle([60, 60])
greedy_search(map2,'manhattan')

In [ ]:
map2, info=generateMap2d_obstacle([60, 60])
greedy_search(map2,'euclidian')

In [ ]:
#Run the program 20 times Greedy search with Euclidean distance and No Obstacle
import time
import numpy
expanded_nodes=[]
path_length=[]
time_taken=[]

for i in range(20):
    map1 = generateMap2d([60, 60])
    start=time.time()
    exp_cntr, path_greedy=greedy_search(map1, 'euclidian')
    stop=time.time()
    expanded_nodes.append(exp_cntr)
    path_length.append(path_greedy)
    timed=stop-start
    time_taken.append(timed)
    
expanded_nodes_mean= numpy.mean(expanded_nodes)
path_mean= numpy.mean(path_length)
time_mean= numpy.mean(time_taken)
expanded_nodes_std_dev = numpy.std(expanded_nodes)
path_mean_std_dev= numpy.std(path_length)
time_mean_std_dev= numpy.std(time_taken)

print(expanded_nodes)
print(path_length)
print(time_taken)
print(expanded_nodes_mean)
print(path_mean)
print(time_mean)
print(expanded_nodes_std_dev)
print(path_mean_std_dev)
print(time_mean_std_dev)



In [ ]:
#Run the program 20 times Greedy search with Manhattan distance and No Obstacle
import time
import numpy
expanded_nodes=[]
path_length=[]
time_taken=[]

for i in range(20):
    map1 = generateMap2d([60, 60])
    start=time.time()
    exp_cntr, path_greedy=greedy_search(map1, 'manhattan')
    stop=time.time()
    expanded_nodes.append(exp_cntr)
    path_length.append(path_greedy)
    timed=stop-start
    time_taken.append(timed)
    
expanded_nodes_mean= numpy.mean(expanded_nodes)
path_mean= numpy.mean(path_length)
time_mean= numpy.mean(time_taken)
expanded_nodes_std_dev = numpy.std(expanded_nodes)
path_mean_std_dev= numpy.std(path_length)
time_mean_std_dev= numpy.std(time_taken)

print(expanded_nodes)
print(path_length)
print(time_taken)
print(expanded_nodes_mean)
print(path_mean)
print(time_mean)
print(expanded_nodes_std_dev)
print(path_mean_std_dev)
print(time_mean_std_dev)



In [ ]:
#Run the program 20 times Greedy search with manhattan distance with Obstacle
import time
import numpy
expanded_obstacle_ctr=[]
path_obstacle_taken=[]
time_obstacle_taken=[]

for i in range(20):
    map3, info=generateMap2d_obstacle([60, 60])
    start=time.time()
    exp_obstacle_nodes, path_greedy=greedy_search(map3, 'manhattan')
    stop=time.time()
    #print(type(exp_cntr),type(path))
    expanded_obstacle_ctr.append(exp_obstacle_nodes)
    path_obstacle_taken.append(path_greedy)
    timed=stop-start
    time_obstacle_taken.append(timed) 


expanded_nodes_obstacle_mean= numpy.mean(expanded_obstacle_ctr)
path_obstacle_mean= numpy.mean(path_obstacle_taken)
time_obstacle_mean= numpy.mean(time_obstacle_taken)
expanded_nodes_obstacle_std_dev = numpy.std(expanded_obstacle_ctr)
path_mean_obstacle_std_dev= numpy.std(path_obstacle_taken)
time_mean_obstacle_std_dev= numpy.std(time_obstacle_taken)

print(expanded_obstacle_ctr)
print(path_obstacle_taken)
print(time_obstacle_taken)
print(expanded_nodes_obstacle_mean)
print(path_obstacle_mean)
print(time_obstacle_mean)
print(expanded_nodes_obstacle_std_dev)
print(path_mean_obstacle_std_dev)
print(time_mean_obstacle_std_dev)



In [ ]:
#Run the program 20 times Greedy search with eclidian distance with Obstacle

import time
import numpy
expanded_obstacle_ctr=[]
path_obstacle_taken=[]
time_obstacle_taken=[]

for i in range(20):
    map4, info=generateMap2d_obstacle([60, 60])
    start=time.time()
    exp_obstacle_nodes1, path_greedy=greedy_search(map4,'euclidian')
    stop=time.time()
    #print(type(exp_cntr),type(path))
    expanded_obstacle_ctr.append(exp_obstacle_nodes1)
    path_obstacle_taken.append(path_greedy)
    timed=stop-start
    time_obstacle_taken.append(timed) 


expanded_nodes_obstacle_mean= numpy.mean(expanded_obstacle_ctr)
path_obstacle_mean= numpy.mean(path_obstacle_taken)
time_obstacle_mean= numpy.mean(time_obstacle_taken)
expanded_nodes_obstacle_std_dev = numpy.std(expanded_obstacle_ctr)
path_mean_obstacle_std_dev= numpy.std(path_obstacle_taken)
time_mean_obstacle_std_dev= numpy.std(time_obstacle_taken)

print(expanded_obstacle_ctr)
print(path_obstacle_taken)
print(time_obstacle_taken)
print(expanded_nodes_obstacle_mean)
print(path_obstacle_mean)
print(time_obstacle_mean)
print(expanded_nodes_obstacle_std_dev)
print(path_mean_obstacle_std_dev)
print(time_mean_obstacle_std_dev)